Equations are given in the format A / B = k, where A and B are variables represented as strings, and k is a real number (floating point number). Given some queries, return the answers. If the answer does not exist, return -1.0.

Example:
Given a / b = 2.0, b / c = 3.0.
queries are: a / c = ?, b / a = ?, a / e = ?, a / a = ?, x / x = ? .
return [6.0, 0.5, -1.0, 1.0, -1.0 ].

The input is: vector<pair<string, string>> equations, vector<double>& values, vector<pair<string, string>> queries , where equations.size() == values.size(), and the values are positive. This represents the equations. Return vector<double>.

According to the example above:

equations = [ ["a", "b"], ["b", "c"] ],
values = [2.0, 3.0],
queries = [ ["a", "c"], ["b", "a"], ["a", "e"], ["a", "a"], ["x", "x"] ]. 

 

The input is always valid. You may assume that evaluating the queries will result in no division by zero and there is no contradiction.


# BFS without caching: O(m(q+n)) ~ O(m * n^2) runtime, O(q) space if there are m queries, q equations and n distinct variables

In [8]:
import collections

from typing import List
from collections import deque

class Solution:
    def calcEquation(self, equations: List[List[str]], values: List[float], queries: List[List[str]]) -> List[float]:
        graph = {}
        
        def build_graph(equations, values):
            def add_edge(f, t, value):
                if f in graph:
                    graph[f].append((t, value))
                else:
                    graph[f] = [(t, value)]
            
            for vertices, value in zip(equations, values):
                f, t = vertices
                add_edge(f, t, value)
                add_edge(t, f, 1/value)
        
        def find_path(query):
            b, e = query
            
            if b not in graph or e not in graph:
                return -1.0
                
            q = deque([(b, 1.0)])
            visited = set()
            
            while q:
                front, cur_product = q.popleft()
                if front == e:
                    return cur_product
                visited.add(front)
                for neighbor, value in graph[front]:
                    if neighbor not in visited:
                        q.append((neighbor, cur_product*value))
            
            return -1.0
        
        build_graph(equations, values)
        return [find_path(q) for q in queries]

# BFS with caching - O(q+n) ~ O(n^2) runtime for all the m queries, O(q) space

In [10]:
import collections

from typing import List
from collections import defaultdict, deque

class Solution:
    def calcEquation(self, equations: List[List[str]], values: List[float], queries: List[List[str]]) -> List[float]:
        graph = defaultdict(dict)
        for ([x,y],value) in zip(equations, values):
            graph[x][y] = value
            graph[y][x] = 1/value
        
        def find_prod(s,e):
            if s not in graph or e not in graph:
                return -1.0
            if s==e: return 1.0
            q = deque([(s, 1.0)])
            visited = {s}
            while q:
                n, curr = q.popleft()
                for child,value in graph[n].items():
                    if child in visited:
                        continue
                    nc = curr*value
                    if child == e:
                        return nc
                    graph[s][child] = nc
                    graph[child][s] = 1/nc
                    visited.add(child)
                    q.append((child, nc))
            return -1.0
        
        return [find_prod(s,e) for [s,e] in queries]

# DFS - O(M * N) runtime, O(N) space, where M is the number of queries and N is the number of equations

In [1]:
from typing import List
from collections import defaultdict

class Solution:
    def calcEquation(self, equations: List[List[str]], values: List[float], queries: List[List[str]]) -> List[float]:

        graph = defaultdict(defaultdict)

        def backtrack_evaluate(curr_node, target_node, acc_product, visited):
            visited.add(curr_node)
            ret = -1.0
            neighbors = graph[curr_node]
            if target_node in neighbors:
                ret = acc_product * neighbors[target_node]
            else:
                for neighbor, value in neighbors.items():
                    if neighbor in visited:
                        continue
                    ret = backtrack_evaluate(
                        neighbor, target_node, acc_product * value, visited)
                    if ret != -1.0:
                        break
            visited.remove(curr_node)
            return ret

        # Step 1). build the graph from the equations
        for (dividend, divisor), value in zip(equations, values):
            # add nodes and two edges into the graph
            graph[dividend][divisor] = value
            graph[divisor][dividend] = 1 / value

        # Step 2). Evaluate each query via backtracking (DFS)
        #  by verifying if there exists a path from dividend to divisor
        results = []
        for dividend, divisor in queries:
            if dividend not in graph or divisor not in graph:
                # case 1): either node does not exist
                ret = -1.0
            elif dividend == divisor:
                # case 2): origin and destination are the same node
                ret = 1.0
            else:
                visited = set()
                ret = backtrack_evaluate(dividend, divisor, 1, visited)
            results.append(ret)

        return results

# Union Find - O( (M + N) * Log N) runtime, O(N) space, where M is the number of queries and N is the number of equations

In [6]:
from typing import List

class Solution:
    def calcEquation(self, equations: List[List[str]], values: List[float], queries: List[List[str]]) -> List[float]:
        gid_weight = {}

        def find(node_id):
            if node_id not in gid_weight:
                gid_weight[node_id] = (node_id, 1)
            group_id, node_weight = gid_weight[node_id]
            if group_id != node_id:
                new_group_id, group_weight = find(group_id)
                gid_weight[node_id] = (new_group_id, group_weight * node_weight)
            return gid_weight[node_id]

        def union(dividend, divisor, value):
            dividend_gid, dividend_weight = find(dividend)
            divisor_gid, divisor_weight = find(divisor)

            if dividend_gid != divisor_gid:
                gid_weight[dividend_gid] = (divisor_gid, divisor_weight * value / dividend_weight)

        for (dividend, divisor), value in zip(equations, values):
            union(dividend, divisor, value)

        res = []
        for dividend, divisor in queries:
            if dividend not in gid_weight or divisor not in gid_weight:
                res.append(-1.0)
            else:
                dividend_group, dividend_weight = find(dividend)
                divisor_group, divisor_weight = find(divisor)

                if dividend_group != divisor_group:
                    res.append(-1.0)
                else:
                    res.append(dividend_weight / divisor_weight)

        return res  

In [8]:
instance = Solution()
instance.calcEquation([["a","b"],["b","c"]],
[2.0,3.0],
[["a","c"],["b","a"],["a","e"],["a","a"],["x","x"]])

[6.0, 0.5, -1.0, 1.0, -1.0]